## 检测python版本
- 这里我们使用的python的版本为3.6.5
- 我们对于将要使用到到库，在这里统一导入，以避免覆盖和名次空间污染

In [1]:
import sys
import keras
import pandas as pd

sys.version

'3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

## 数据预处理
- 下载好的数据集位于data目录下
- 筛选出训练集中不合格到图片，在pick_bad_pics.ipynb中说明并实现
- 将数据集解压后，归类图片到指定到目录（目录结构readme.md中有描述）
- 本note不对数据到预处理做重复描述

In [3]:
# 数据预处理，见其它notebook和附件

## 模型对比、筛选和验证
- 我们将编写函数，将需要采用到到预先训练模型，分部进行训练和预测。根据其中到表现，筛选出我们最终会使用到预训练模型
- 我们将对比和验证到模型有：InceptionV3、Xception和Inception ResnetV2

In [ ]:
# 预训练模型筛选

# 构建模型
def buid_model(pre_model, image_size, pre_input):

    # 获取基础模型，不保留顶层的全连接网络
    input_tensor = keras.Input(shape=(image_size[0], image_size[1], 3))
    if pre_input:
        input_tensor = keras.layers.Lambda(pre_input)(input_tensor)
    base_model = pre_model(input_tensor=input_tensor, include_top=False)

    # 锁定模型，保护处理
    for layer in base_model.layers:
        layer.trainable = False

    # 空域信号施加全局平均池化，dropout处理防止过拟合，重建全连接层
    tmp = keras.layers.GlobalAveragePooling2D()(base_model.output)
    tmp = keras.layers.Dropout(0.25)(tmp)
    tmp = keras.layers.Dense(1, activation='sigmoid', kernel_initializer='he_normal')(tmp)

    # 配置模型
    model_obj = keras.models.Model(inputs=base_model.input, outputs=x)
    model_obj.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

    # 返回
    print('%s has %d layers.' % (model_obj.__name__, len(model_obj.layers)))
    return model_obj


# 预测输出，并保存结果
def do_predict(model_obj, image_size):

    # 定义图片生成器
    gen = keras.preprocessing.image.ImageDataGenerator()
    train_generator = gen.flow_from_directory("data/train_gen", image_size, shuffle=False)
    test_generator  = gen.flow_from_directory("data/test", image_size, shuffle=False, class_mode=None)

    # 训练
    model_obj.fit_generator(train_generator, len(train_generator), epochs=8, verbose=2)
 
    
    # 预测输出
    pred = model_obj.predict_generator(test_gen, verbose=1)
    pred = pred.clip(min=0.005, max=0.995)

    df = pd.read_csv("sample_submission.csv")
    for i, fname in enumerate(test_gen.filenames):
        index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
        df.set_value(index-1, 'label', y_pred[i])

    # 保存结果
    df.to_csv('submission_%s.csv' % model_obj.__name__, index=None)
    df.head(20)
    
# 筛选测试
def select_best_model(pre_model, image_size, pre_input):
    
    # 构建模型
    model_obj = buid_model(pre_model, image_size, pre_input)
    
    # 预测输出
    do_predict(model_obj, image_size)

## 模型InceptionV3
预训练模型InceptionV3

In [ ]:
select_best_model(keras.applications.inception_v3.InceptionV3, (299, 299)
                  , keras.applications.inception_v3.preprocess_input)

## 模型Xception
预训练模型Xception

In [ ]:
select_best_model(keras.applications.xception.Xception, (299, 299)
                  , keras.applications.xception.preprocess_input)

## 模型Inception ResnetV2
预训练模型Inception ResnetV2

In [ ]:
select_best_model(keras.applications.inception_resnet_v2.InceptionResNetV2, (299, 299)
                  , keras.applications.inception_resnet_v2.preprocess_input)